In [7]:
import pandas as pd
import numpy as np
import cv2

from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from keras.models import Sequential, model_from_json
from keras.layers import Conv2D, MaxPooling2D, Input, AveragePooling2D, Flatten, Dense, Dropout, ELU, Activation
from keras.optimizers import SGD, Adam
from keras.regularizers import l2

### Get data

In [2]:
# load driving log data
driving_log = pd.read_csv('data/driving_log.csv', sep=',')
# use left and right cameras images when steering is not zero
driving_log_steer_not_0 = driving_log[driving_log.steering != 0]

input_images = []
labels_list = []

avg_steer = np.mean(driving_log.steering.tolist())
ext_steer = 0.5
rep_size = 5
ext_size = 30
offset = 0.2

# load center images
images_list = list(driving_log.center)
for i in range(0, len(images_list)):
    
    img = np.asarray(Image.open('data/%s' % images_list[i]))
    steer = driving_log.steering[i]
    
    # upsize images with extreme steering 
    if abs(steer) > ext_steer:
        for j in range(0, ext_size):
            input_images.append(img)
            labels_list.append(steer)
    
    # upsize images with steering bigger than averagge level
    elif abs(steer) > avg_steer:
        for j in range(0, rep_size):
            input_images.append(img)
            labels_list.append(steer)
    
    # for zero steering or small steering, just load one time the image
    else:
        input_images.append(img)
        labels_list.append(steer)
    
# load side images 
side_images_list = list(driving_log_steer_not_0.left) + list(driving_log_steer_not_0.right)
for img_name in side_images_list:
    img_name = img_name[1:]
    img = np.asarray(Image.open('data/%s' % img_name))
    input_images.append(img)

# load side labels
steering_left_image = [x + offset for x in driving_log_steer_not_0.steering]
steering_right_image = [x - offset for x in driving_log_steer_not_0.steering]
labels_list = labels_list + steering_left_image + steering_right_image
    
# convert to array format
input_images = np.asarray(input_images)
print(input_images.shape)
output_labels = np.asarray(labels_list)
print(output_labels.shape)

(30650, 160, 320, 3)
(30650,)


### Preprocess data

In [3]:
# normalization
input_images = input_images.astype('float32')
input_images = input_images / 128 - 0.5

# resize images
ch, row, col = 3, 66, 200
input_images_resized = []
for img in input_images:
    img_resized = cv2.resize(img, (col, row))
    input_images_resized.append(img_resized)
    
input_images_resized = np.asarray(input_images_resized)
print(input_images_resized.shape)

(30650, 66, 200, 3)


### Split train & test data

In [4]:
X_train, X_test, y_train, y_test = train_test_split(input_images_resized, output_labels, test_size=0.2, random_state=0)

### Create Keras Model

In [ ]:
def creat_model_commaAI():
    
    model = Sequential()

    model.add(Conv2D(16, 8, 8, input_shape=(row, col, ch), subsample=(4, 4), border_mode="same"))
    model.add(ELU())
    
    model.add(Conv2D(32, 5, 5, subsample=(2, 2), border_mode="same"))
    model.add(ELU())
    
    model.add(Conv2D(64, 5, 5, subsample=(2, 2), border_mode="same"))
    
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(ELU())
    
    model.add(Dense(512))
    model.add(Dropout(0.5))
    model.add(ELU())
    
    model.add(Dense(1))

    model.compile(optimizer=Adam(lr=0.0001), loss="mse")

    return model

In [17]:
def creat_model_nvidia():
    
    model = Sequential()

    model.add(Conv2D(24, 5, 5, input_shape=(row, col, ch), subsample=(2, 2), border_mode="valid", activation='relu'))
    model.add(Conv2D(36, 5, 5, subsample=(2, 2), border_mode="valid", activation='relu'))
    model.add(Conv2D(48, 5, 5, subsample=(2, 2), border_mode="valid", activation='relu'))
    model.add(Conv2D(64, 3, 3, subsample=(1, 1), border_mode="valid", activation='relu'))
    model.add(Conv2D(64, 3, 3, subsample=(1, 1), border_mode="valid", activation='relu'))
    
    model.add(Flatten())
    
    model.add(Dense(1164))
    model.add(Dense(100))
    model.add(Dense(50))
    model.add(Dense(10))
    model.add(Dense(1))

    model.compile(optimizer=Adam(lr=0.0001), loss="mse")

    return model

In [21]:
def creat_model_nvidia():
    
    model = Sequential()

    model.add(Conv2D(24, 5, 5, input_shape=(row, col, ch), subsample=(2, 2), border_mode="valid", activation='relu'))
    model.add(Conv2D(36, 5, 5, subsample=(2, 2), border_mode="valid", activation='relu'))
    model.add(Conv2D(48, 5, 5, subsample=(2, 2), border_mode="valid"))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    
    model.add(Conv2D(64, 3, 3, subsample=(1, 1), border_mode="valid", activation='relu'))
    model.add(Conv2D(64, 3, 3, subsample=(1, 1), border_mode="valid"))
    model.add(Dropout(0.5))
    model.add(Activation('relu'))
        
    model.add(Flatten())
    
    model.add(Dense(1164))
    model.add(Dropout(0.5))
    model.add(Activation('relu'))
    
    model.add(Dense(100))
    model.add(Dense(50))
    model.add(Dense(10))
    model.add(Dense(1))

    model.compile(optimizer=Adam(lr=0.00001), loss="mse")

    return model

### Train model

In [22]:
model = creat_model_nvidia()
model.summary()

history = model.fit(X_train, y_train,
                    batch_size=128, nb_epoch=20,
                    verbose=1, shuffle=True,
                    validation_data=(X_test, y_test))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_41 (Convolution2D) (None, 31, 98, 24)    1824        convolution2d_input_9[0][0]      
____________________________________________________________________________________________________
convolution2d_42 (Convolution2D) (None, 14, 47, 36)    21636       convolution2d_41[0][0]           
____________________________________________________________________________________________________
convolution2d_43 (Convolution2D) (None, 5, 22, 48)     43248       convolution2d_42[0][0]           
____________________________________________________________________________________________________
dropout_8 (Dropout)              (None, 5, 22, 48)     0           convolution2d_43[0][0]           
___________________________________________________________________________________________

### Save model and weights

In [23]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model.h5")

print("Model and weights saved!")

Model and weights saved!
